### Datasets
http://www.manythings.org/anki/  (Download and unzip mar-eng.zip file)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
!git clone https://github.com/wl-research/nubia.git
import os
os.chdir('nubia')

Cloning into 'nubia'...
remote: Enumerating objects: 272, done.
remote: Total 272 (delta 0), reused 0 (delta 0), pack-reused 272
Receiving objects: 100% (272/272), 70.09 MiB | 27.55 MiB/s, done.
Resolving deltas: 100% (120/120), done.


In [2]:
!pip install -r requirements.txt

     |████████████████████████████████| 307kB 3.4MB/s 
     |████████████████████████████████| 184kB 8.2MB/s 
     |████████████████████████████████| 61kB 5.9MB/s 
     |████████████████████████████████| 890kB 10.9MB/s 
     |████████████████████████████████| 1.1MB 19.1MB/s 
  Created wheel for fairseq: filename=fairseq-0.9.0-cp36-cp36m-linux_x86_64.whl size=2021167 sha256=14f300b0a3a5c5d0adf78bd755011d4e404bbdffe9e734cb17a9379d1c26d0f2
  Stored in directory: /root/.cache/pip/wheels/37/3e/1b/0fa30695dcba41e4b0088067fa40f3328d1e8ee78c22cd4766
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=50eca604d1b9e81c1000dc596c6f687901d4a1586969b91b26a8623b5c9db0f0
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=d122872b73dbde1399dc7be3a1ede654a6d9cf2544d3336f385658159f7be710
  Stored in directory: /root/.cache/pip/

In [3]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from nubia import Nubia

Using TensorFlow backend.


In [4]:
nubia = Nubia()

Streaming output truncated to the last 5000 lines.
Downloading: 94% [775020544 / 815943985] bytes
Downloading: 94% [775028736 / 815943985] bytes
Downloading: 94% [775036928 / 815943985] bytes
Downloading: 94% [775045120 / 815943985] bytes
Downloading: 94% [775053312 / 815943985] bytes
Downloading: 94% [775061504 / 815943985] bytes
Downloading: 94% [775069696 / 815943985] bytes
Downloading: 94% [775077888 / 815943985] bytes
Downloading: 94% [775086080 / 815943985] bytes
Downloading: 94% [775094272 / 815943985] bytes
Downloading: 94% [775102464 / 815943985] bytes
Downloading: 94% [775110656 / 815943985] bytes
Downloading: 94% [775118848 / 815943985] bytes
Downloading: 94% [775127040 / 815943985] bytes
Downloading: 94% [775135232 / 815943985] bytes
Downloading: 94% [775143424 / 815943985] bytes
Downloading: 95% [775151616 / 815943985] bytes
Downloading: 95% [775159808 / 815943985] bytes
Downloading: 95% [775168000 / 815943985] bytes
Downloading: 95% [775176192 / 815943985] bytes
Downloadi

1042301B [00:00, 2332690.14B/s]
456318B [00:00, 1431170.96B/s]


loading archive file pretrained/roBERTa_MNLI
| dictionary: 50264 types


100%|██████████| 548118077/548118077 [00:08<00:00, 67053105.08B/s]


In [0]:
lines= pd.read_table('/content/drive/My Drive/rus.txt', names=['eng','rus', '1'])

In [0]:
lines = lines.drop(['1'], axis=1)
lines = lines[:38696]
lines = shuffle(lines)

In [7]:
lines.shape

(38696, 2)

In [0]:
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.rus=lines.rus.apply(lambda x: x.lower())

# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.rus=lines.rus.apply(lambda x: re.sub("'", '', x))

exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.rus=lines.rus.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng = lines.eng.apply(lambda x: x.translate(remove_digits))
lines.rus = lines.rus.apply(lambda x: x.translate(remove_digits))

# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.rus=lines.rus.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.rus=lines.rus.apply(lambda x: re.sub(" +", " ", x))

# Add start and end tokens to target sequences
lines.eng = lines.eng.apply(lambda x : 'START_ '+ x + ' _END')

In [14]:
lines.sample(10)

,eng,rus
14839,START_ i love my kids _END,я люблю своих детей
19092,START_ call the police _END,вызывай полицию
15279,START_ ill ask later _END,я потом спрошу
4270,START_ ignore that _END,не обращай на это внимания
22967,START_ no ones hungry _END,никто не голоден
15321,START_ ill go myself _END,я сама пойду
17997,START_ we were framed _END,нас подставили
26070,START_ youre a person _END,ты человек
3870,START_ i said stop _END,я сказал хватит
19301,START_ did tom go home _END,том домой пошёл


In [0]:
# Vocabulary of Russian
all_rus_words=set()
for rus in lines.rus:
    for word in rus.split():
        if word not in all_rus_words:
            all_rus_words.add(word)

# Vocabulary of English 
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

In [16]:
# Max Length of source sequence
lenght_list=[]
for l in lines.rus:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

10

In [17]:
# Max Length of target sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

8

In [18]:
input_words = sorted(list(all_rus_words))
target_words = sorted(list(all_eng_words))
num_encoder_tokens = len(all_rus_words)
num_decoder_tokens = len(all_eng_words)
num_encoder_tokens, num_decoder_tokens

(11582, 4207)

In [19]:
num_decoder_tokens += 1 # For zero padding
num_encoder_tokens +=1
num_decoder_tokens

4208

In [0]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [0]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [22]:
lines.head(10)

,eng,rus
27222,START_ does tom have it _END,у тома оно есть
36828,START_ everything is new _END,всё новое
13790,START_ dont leave me _END,не бросай меня
30592,START_ it isnt a dream _END,это не сон
33039,START_ tom is exhausted _END,том обессилен
28341,START_ i cant go alone _END,я не могу пойти одна
20323,START_ i can sing too _END,я тоже могу петь
4658,START_ she fainted _END,она упала в обморок
24477,START_ tom knew no one _END,том ни с кем знаком не был
11632,START_ take that one _END,возьми ту


In [23]:
# Train - Test Split
X, y = lines.rus, lines.eng
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((34826,), (3870,))

#### Save the train and test dataframes for reproducing the results later, as they are shuffled.

In [0]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder - Decoder Model Architecture

In [0]:
latent_dim = 50

In [0]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [0]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [0]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 32
epochs = 1

In [0]:
model.load_weights('/content/drive/My Drive/Colab Notebooks/weights.h5')

In [33]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = 200,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
200/200 [==============================] - 19s 97ms/step - loss: 0.9632 - acc: 0.7605 - val_loss: 0.8979 - val_acc: 0.7588


In [0]:
model.save_weights('weights.h5')

### Inference Setup

In [0]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

### Decode sample sequeces

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

### Evaluation on Train Dataset

In [0]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [66]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_train[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_train[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_train[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: том неискренний
Actual English translation:  tom is deceitful 
Predicted English translation:  it was tom 
Semantic relation: 0.7205052375793457/5.0
Percent chance of contradiction: 26.138517260551453%
Percent chance of irrelevancy or new information: 63.5387659072876%
Percent chance of logical agreement: 10.322712361812592%

NUBIA score: 0.46372482305638885/1.0


0.46372482305638885

In [49]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_train[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_train[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_train[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: пожалуйста прекрати это
Actual English translation:  please stop it 
Predicted English translation:  its a lot 
Semantic relation: 1.555252194404602/5.0
Percent chance of contradiction: 1.7395731061697006%
Percent chance of irrelevancy or new information: 66.66330695152283%
Percent chance of logical agreement: 31.59712255001068%

NUBIA score: 0.29142747307207184/1.0


0.29142747307207184

In [52]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_train[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_train[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_train[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: стопудово
Actual English translation:  yes of course 
Predicted English translation:  he was up 
Semantic relation: 1.980818510055542/5.0
Percent chance of contradiction: 2.031507156789303%
Percent chance of irrelevancy or new information: 91.29781126976013%
Percent chance of logical agreement: 6.670675426721573%

NUBIA score: 0.2985636073411566/1.0


0.2985636073411566

In [69]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_train[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_train[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_train[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: том утвердительно кивнул
Actual English translation:  tom nodded ok 
Predicted English translation:  you need to rest 
Semantic relation: 1.2755951881408691/5.0
Percent chance of contradiction: 0.6392671260982752%
Percent chance of irrelevancy or new information: 99.14755821228027%
Percent chance of logical agreement: 0.21317268256098032%

NUBIA score: 0.636071633401742/1.0


0.636071633401742

### Evaluation on Validation Dataset

In [0]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [74]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_test[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_test[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_test[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: том понял
Actual English translation:  tom understood 
Predicted English translation:  tom got me 
Semantic relation: 0.8201012015342712/5.0
Percent chance of contradiction: 0.5283449776470661%
Percent chance of irrelevancy or new information: 18.161186575889587%
Percent chance of logical agreement: 81.31047487258911%

NUBIA score: 0.6968490218583063/1.0


0.6968490218583063

In [76]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_test[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_test[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_test[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: что ты сказал
Actual English translation:  what did you say 
Predicted English translation:  what did you say 
Semantic relation: 4.840568542480469/5.0
Percent chance of contradiction: 1.623729057610035%
Percent chance of irrelevancy or new information: 1.166675891727209%
Percent chance of logical agreement: 97.20959663391113%

NUBIA score: 1.0/1.0


1.0

In [78]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_test[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_test[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_test[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: том ошибался
Actual English translation:  tom was wrong 
Predicted English translation:  tom was wrong 
Semantic relation: 3.431398868560791/5.0
Percent chance of contradiction: 0.1719208201393485%
Percent chance of irrelevancy or new information: 1.0978682897984982%
Percent chance of logical agreement: 98.73020648956299%

NUBIA score: 1.0/1.0


1.0

In [80]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_test[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_test[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_test[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: мы заблудились
Actual English translation:  we got lost 
Predicted English translation:  were lost 
Semantic relation: 3.7291259765625/5.0
Percent chance of contradiction: 0.26536115910857916%
Percent chance of irrelevancy or new information: 1.0849418118596077%
Percent chance of logical agreement: 98.64969849586487%

NUBIA score: 0.5906952187102021/1.0


0.5906952187102021

In [83]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_test[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_test[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_test[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: у него не было денег
Actual English translation:  he had no money 
Predicted English translation:  he had no money 
Semantic relation: 4.994417190551758/5.0
Percent chance of contradiction: 0.2987137297168374%
Percent chance of irrelevancy or new information: 0.9756773710250854%
Percent chance of logical agreement: 98.72560501098633%

NUBIA score: 1.0/1.0


1.0

In [84]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_test[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_test[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_test[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: они доктора
Actual English translation:  theyre doctors 
Predicted English translation:  they are a lot 
Semantic relation: 1.435977816581726/5.0
Percent chance of contradiction: 4.46191169321537%
Percent chance of irrelevancy or new information: 89.06938433647156%
Percent chance of logical agreement: 6.468704342842102%

NUBIA score: 0.32035303384572267/1.0


0.32035303384572267

In [85]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input Russian sentence:' +'\033[0m', X_test[k:k+1].values[0])
print('\033[1m'+ 'Actual English translation:' +'\033[0m', y_test[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted English translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_test[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input Russian sentence: как том мог выиграть
Actual English translation:  how could tom win 
Predicted English translation:  how did tom win 
Semantic relation: 4.144294738769531/5.0
Percent chance of contradiction: 1.8374454230070114%
Percent chance of irrelevancy or new information: 2.0112859085202217%
Percent chance of logical agreement: 96.15126848220825%

NUBIA score: 0.8534810744294751/1.0


0.8534810744294751